In [1]:
username = "Alex"
pathtorepo = "C:\\Users\\" *username *  "\\Desktop\\"
using Pkg
Pkg.activate(pathtorepo * "dynamical-systems\\env\\integrate\\")
include(pathtorepo * "dynamical-systems\\system.jl")


using StaticArrays, DifferentialEquations, DynamicalSystems, JLD
E, x, y  = -40..40, -5..5,  -5..5
box = E × x × y
using GLMakie
using LinearAlgebra, LinearSolve,Statistics

cd("C:\\Users\\" *username *  "\\Desktop\\dynamical-systems\\Tsodyks Markram\\Levanova\\3 набор параметров\\Сопоставление с матконт\\файлы matlab")
I0_hom = load("I0_hom_hom.jld")["data"]
u0_hom = load("U0_hom_hom.jld")["data"]
I0_hom = I0_hom[:]
u0_hom = u0_hom[:];

  Activating 

project at `C:\Users\Alex\Desktop\dynamical-systems\env\integrate`


In [2]:
function condition(u, t, integrator)
    x = Vector(u)
    b = x - fp[1]
    b = Vector(b)
    linprob = LinearProblem(A, b)
    linsolve = solve(linprob)
    return norm(linsolve.u, Inf) - ϵ_box
end
affect!(integrator) = terminate!(integrator)
function get_norm_αs(u)
    x = Vector(u)
    b = x - fp[1]
    b = Vector(b)
    linprob = LinearProblem(A, b)
    linsolve = solve(linprob)
    return norm(linsolve.u, Inf), linsolve
end
function color_alpha(alpha)
    if alpha > 0.0
        return "red"
    else
        return "blue"
    end
end;

In [3]:
const τ = 0.013;  const τD = 0.07993;  const τy = 3.3;  const J = 3.07;  const β = 0.300
const xthr = 0.75; const ythr = 0.4; const α = 1.58; const ΔU0 = 0.305

t = 2000.0; tspan = [0.0, t];

In [4]:
# index point from homoclinic curve
index_hom = 1
I0 = -1.7062; U0 = u0_hom[index_hom]

p = SA[α, τ, τD, τy, J, xthr, ythr, U0, ΔU0, β, I0]
u0 = [0.0, 0.0, 0.0]

ds = CoupledODEs(TM, u0, p)
fp, _, _ = fixedpoints(ds, box, jacob_TM_)

ϵ_box = 1e-5
Ju0 = jacob_TM_(fp[1], p, 0);
fp_ = fp[1]

eigen_val_vec = eigen(Ju0)
eigen_vectors = eigen_val_vec.vectors

v1 = real(eigen_vectors[:, 1])
v2 = real(eigen_vectors[:, 2])
v3 = imag(eigen_vectors[:, 3])

v1 = normalize(v1, 2)
v1t = transpose(v1)

v2 = normalize(v2, 2)
v2t = transpose(v2)

v3 = normalize(v3, 2)
v3t = transpose(v3)

A = transpose([v1t; v2t; v3t])
A = Matrix(A)

3×3 Matrix{Float64}:
  0.998728    0.999899     0.0
 -0.0489909  -0.014181     0.9847
  0.011912    0.000705909  0.174257

In [5]:
total_number_points = 50
number_points_on_side = Int((total_number_points) / 1)

dots_right_side = zeros(number_points_on_side, length(u0))
array_α_vec_right = zeros(number_points_on_side, length(u0))

dots_left_side = zeros(number_points_on_side, length(u0))
array_α_vec_left = zeros(number_points_on_side, length(u0))

dots_up_side = zeros(number_points_on_side, length(u0))
array_α_vec_up = zeros(number_points_on_side, length(u0))

dots_down_side = zeros(number_points_on_side, length(u0))
array_α_vec_down = zeros(number_points_on_side, length(u0));

In [6]:
# case α1 = 0; α2 = ϵ_box; α3 = [-ϵ_box; ϵ_box]
n = 1
N = total_number_points
while n <= number_points_on_side

    α1 = 0.0
    α2 = ϵ_box
    α3 = - ϵ_box + 2 * ϵ_box * n / N
    α_vec = [α1, α2, α3]
    point = fp_ + A * α_vec
    
    dots_right_side[n, :] = point
    _, α_vec = get_norm_αs( dots_right_side[n, :] )
    array_α_vec_right[n, :] = α_vec
    n+=1
end

# case α1 = 0; α2 = -ϵ_box; α3 = [-ϵ_box; ϵ_box]
n = 1
N = total_number_points
while n <= number_points_on_side

    α1 = 0.0
    α2 = -ϵ_box
    α3 = - ϵ_box + 2 * ϵ_box * n / N
    α_vec = [α1, α2, α3]
    point = fp_ + A * α_vec
    
    dots_left_side[n, :] = point
    _, α_vec = get_norm_αs( dots_left_side[n, :] )
    array_α_vec_left[n, :] = α_vec
    n+=1
end

# case α1 = 0; α2 = [-ϵ_box; ϵ_box]; α3 = ϵ_box
n = 1
N = total_number_points
while n <= number_points_on_side

    α1 = 0.0
    α2 = - ϵ_box + 2 * ϵ_box * n / N
    α3 = ϵ_box
    α_vec = [α1, α2, α3]
    point = fp_ + A * α_vec
    
    dots_up_side[n, :] = point
    _, α_vec = get_norm_αs( dots_up_side[n, :] )
    array_α_vec_up[n, :] = α_vec
    n+=1
end

# case α1 = 0; α2 = [-ϵ_box; ϵ_box]; α3 = -ϵ_box
n = 1
N = total_number_points
while n <= number_points_on_side

    α1 = 0.0
    α2 = - ϵ_box + 2 * ϵ_box * n / N
    α3 = -ϵ_box
    α_vec = [α1, α2, α3]
    point = fp_ + A * α_vec
    
    dots_down_side[n, :] = point
    _, α_vec = get_norm_αs( dots_down_side[n, :] )
    array_α_vec_down[n, :] = α_vec
    n+=1
end;

In [7]:
Es = Float64[]; xs = Float64[]; ys = Float64[]
α1s =  Float64[]; α2s =  Float64[]; α3s =  Float64[]

colors = []
check_events = []
time_events = []

Any[]

In [8]:
function trajectory_from_side(dots_from_side, number_points_on_side, cb)
    
    for index in range(1, number_points_on_side, step = 1)
    
        u0 = SA[dots_from_side[index, 1], dots_from_side[index, 2], dots_from_side[index, 3]]
        prob = ODEProblem(TM, u0, tspan, p)
        sol = solve(prob, alg = Vern9(), abstol = 1e-14, reltol = 1e-14, callback = cb)
        norm, linsolve = get_norm_αs(sol[end])
    
        if sol.retcode == ReturnCode.Terminated
            push!(Es, sol[end][1])
            push!(xs, sol[end][2])
            push!(ys, sol[end][3])
            push!(α1s, linsolve[1])
            push!(α2s, linsolve[2])
            push!(α3s, linsolve[3])
            push!(colors, color_alpha(linsolve[1]))
            push!(check_events, true)
            push!(time_events, sol.t[end])
        else
           push!(check_events, false)
        end
    end

end

trajectory_from_side (generic function with 1 method)

In [9]:
cb = ContinuousCallback(condition, nothing, affect!)
trajectory_from_side(dots_up_side, number_points_on_side, cb)
trajectory_from_side(dots_down_side, number_points_on_side, cb)

In [ ]:
number_of_event = length(findall(check_events .== true))
number_of_no_event = length(check_events) - number_of_event
println("number of event: $(number_of_event); number of no event: $(number_of_no_event)")

In [ ]:
number_of_event = length(findall(check_events .== true))
number_of_no_event = length(check_events) - number_of_event
println("number of event: $(number_of_event); number of no event: $(number_of_no_event)")

In [ ]:
# last points in space (α2, α3)
f = Figure(resolution = (700, 700))

ax = Axis(f[1, 1], xlabel = L"α_2", ylabel = L"α_3", xlabelsize = 35, ylabelsize = 35)
idx, idy = 5, 6

scatter!(ax, 0.0, 0.0, color = :green, markersize = 15, marker = :rect)

scatter!(ax, α2s, α3s, color = colors[:], markersize = 5)


display(GLMakie.Screen(), f);

In [ ]:
# last points in phase space
idx, idy, idz = 2, 3, 1

f = Figure(resolution = (700, 700))

ax = LScene(f[1, 1], show_axis = true)
    
scale!(ax.scene, 50, 50, 1)

scatter!(xs, ys, Es, markersize = 7.0, color = :deeppink)

scatter!(fp[1][idx], fp[1][idy], fp[1][idz], color = :blue, markersize = 7.0)

display(GLMakie.Screen(), f);